### Use Embeddings extracted from Bio2RDF for Drugs

In [1]:
!pip install progressbar2

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd
import pickle
import numpy as np
import torch
from progressbar import ProgressBar

C:\Users\dlezo\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


#### Load the Drugs common names from DrugBank

In [3]:
with open('./Data/rx_norm_to_db.pkl', 'rb') as file:
    rx_norm_to_db = pickle.load(file)

#### Load the embeddings file

In [4]:
kg_embeddings = pd.read_csv("./Data/Embeddings/Entity2Vec_sg_200_5_5_15_2_500_d5_uniform.txt")
kg_embeddings.head()

,DB,feature0,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,...,feature190,feature191,feature192,feature193,feature194,feature195,feature196,feature197,feature198,feature199
0,DB00142,0.596122,-0.561010,1.195846,-0.848914,0.682194,0.837945,0.328771,-1.415323,0.520460,...,-0.117932,0.937970,0.403538,0.334809,0.204034,-1.107825,0.523416,-0.414221,-0.214047,0.376912
1,DB00130,-0.686141,0.348200,0.163095,-1.113229,0.433326,0.959762,0.224798,-1.213323,0.312421,...,0.493890,0.797126,0.930760,1.718329,0.105792,-1.058102,0.416409,0.380049,-0.423195,1.282005
2,DB00126,-0.057253,-0.029533,-0.920006,-0.581274,0.456974,0.034768,-0.012655,-0.330921,-0.373522,...,-0.310304,1.728169,-0.864270,1.037250,-0.135598,-1.372231,1.117598,0.610883,-0.288994,-0.146524
3,DB00134,0.284603,-0.516267,0.764172,-1.363171,-0.207583,0.815897,0.609548,-1.594437,0.922312,...,0.416226,0.431215,0.035447,1.137347,0.575587,-0.850762,0.456986,0.366332,-0.546449,0.759064
4,DB00150,0.904884,-0.207666,-0.371477,-0.314093,-0.404803,0.998473,0.529165,-1.367542,0.888355,...,-0.052134,0.810354,0.175572,1.083338,-0.462010,-1.330534,0.251962,0.201748,0.375982,-0.124131


#### Get the embedding from Bio2RDF with RDF2Vec & SkipGram

In [5]:
embeddings = {}
i = 0
step = len(rx_norm_to_db.keys())//100
bar = ProgressBar(max_value=100)
for rx_norm_id in rx_norm_to_db.keys():
    if i%step==0:
        bar.next()
    output_embedding = torch.from_numpy(np.array(
        kg_embeddings[kg_embeddings['DB'] == rx_norm_to_db[rx_norm_id]].iloc[0]['feature0':'feature199'].values, dtype=np.float64))
    embeddings[rx_norm_id] = output_embedding
    i += 1
bar.finish()

100% (100 of 100) |######################| Elapsed Time: 0:00:00 Time:  0:00:000000


In [6]:
# Get the embeddings in a list
embeddings768 = []
for rx_norm_id in rx_norm_to_db.keys():
    embeddings768.append(embeddings[rx_norm_id].detach().numpy())
embeddings768 = np.array(embeddings768)

#### Reduce the embeddings dimensionality with PCA

In [7]:
import numpy as np
from sklearn.decomposition import PCA

size = 128
pca = PCA(n_components=size, svd_solver='full')
embeddings_pca = pca.fit_transform(embeddings768)

print('Number of components:', pca.n_components_)
print('Covariance covered:', pca.explained_variance_ratio_.sum())

Number of components: 128
Covariance covered: 0.9513708059797161


In [8]:
embeddings_size_pca = {}
i = 0
for rx_norm_id in embeddings.keys():
    embeddings_size_pca[rx_norm_id] = embeddings_pca[i]
    i += 1

In [10]:
with open('./Data/Embeddings/kg_bio2rdf_embeddings_'+str(size)+'_pca.pkl', 'wb') as file:
    pickle.dump(embeddings_size_pca, file)
    print(file.name)

./Data/TWOSIDES-kg_bio2rdf_embeddings_128_pca.pkl
